In [ ]:
# import libraries
import pandas as pd
import numpy as np
# for distance
from math import radians, cos, sin, asin, sqrt
# For model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
# For csv
from itertools import cycle
#######################################################################################################################
#COMMON FUNCTIONS FOR ALL DATA
#DATE_HRS Dataframe
# Creating a dataframe with the time range
def Date_hrs(min_date, max_date):
    DATE_hrs = pd.DataFrame({"date": pd.date_range(min_date, max_date, freq='H')})
    print("No of Hrs : " +str(DATE_hrs.shape))
    return DATE_hrs

# Reading all the stations file only with lat, long, station name
# Air quality station
AQ_station = pd.read_csv("Beijing_AirQuality_Stations_en.csv")
# Observed weather station
OW_station = pd.read_csv("OW_station_data.csv")
# Grid weather station
GW_station = pd.read_csv("Beijing_grid_weather_station.csv", names = ["Grid_station","latitude","longitude"])
GW_station = GW_station.rename(columns={'Grid_station': 'station'})

#Nearest station
# Finiding nearest station
def haversine_dist(long1, lat1, long2, lat2):
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])
    # haversine formula 
    dist_long = long2 - long1 
    dist_lat = lat2 - lat1 
    a = sin(dist_lat/2)**2 + cos(lat1) * cos(lat2) * sin(dist_long/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 
    return c * r

def near_station(lat, long, station):
    dists = station.apply(lambda row: haversine_dist(lat, long, row['latitude'], row['longitude']), axis=1)
    dist = dists[dists!=0]
    return station.loc[dist.idxmin(), 'station']
# Nearest AQ_station for AQ_station
AQ_station["nearest_AQ"] = AQ_station.apply(
    lambda row: near_station(row['latitude'], row['longitude'],AQ_station), 
    axis=1)
# Nearest OW_station for OW_station
OW_station["nearest_OW"] = OW_station.apply(
    lambda row: near_station(row['latitude'], row['longitude'], OW_station), 
    axis=1)
# Nearest GW_station for GW_station
GW_station["nearest_GW"] = GW_station.apply(
    lambda row: near_station(row['latitude'], row['longitude'], GW_station), 
    axis=1)
# Nearest OW_station for AQ_data
Final_station = AQ_station
Final_station["nearest_OW"] = Final_station.apply(
    lambda row: near_station(row['latitude'], row['longitude'], OW_station), 
    axis=1)
# Nearest GW_station for AQ_data
Final_station["nearest_GW"] = Final_station.apply(
    lambda row: near_station(row['latitude'], row['longitude'], GW_station), 
    axis=1)

#Getting Year, Day, Month, Hour from Date
# Getting day month hour week from time in new_df
def ymd(df):
    df['date'] = pd.to_datetime(df['date'])
    df['year'], df['month'] ,df['hour'], df["week"], df["day"] = df['date'].dt.year, df['date'].dt.month, df['date'].dt.hour, df['date'].dt.week ,df['date'].dt.day
    return df

###############################################################################################################################################################################################################################################################
###############################################################################################################################################################################################################################################################

# TRAIN DATA
# AIR QULAITY DATA

# Reading the air quality data
# Reading the air quality data
AQ_jan17_to_jan18= pd.read_csv('airQuality_201701-201801.csv')
AQ_feb18_march18= pd.read_csv('airQuality_201802-201803.csv')
AQ_april18= pd.read_csv('aiqQuality_201804.csv')
AQ_april18= AQ_april18.drop(['id'], axis=1)
AQ_april18.columns = ['stationId','utc_time','PM2.5','PM10','NO2','CO', 'O3', 'SO2']

# Appending all the air quality data
AQ_data = pd.DataFrame()
AQ_data = AQ_jan17_to_jan18.append([AQ_feb18_march18,AQ_april18])
AQ_data = AQ_data.rename(index=str, columns={"stationId": "station_id", "utc_time": "time"})
AQ_data["time"] = pd.to_datetime(AQ_data['time'], format='%Y-%m-%d %H:%M:%S')

# Checking the shape of AQ_data before removing duplicates
print("Shape of AQ_data before removing the duplicates:", AQ_data.shape)
# Dropping duplicates
AQ_data.drop_duplicates(subset= None, keep= "first", inplace= True)
# Sorting the entire dataframe based on station_id and time
AQ_data = AQ_data.sort_values(by=['station_id', 'time'], ascending=[True,True] )
# Checking the shape of AQ_data after removing duplicates
print("Shape of AQ_data after removing the duplicates:", AQ_data.shape)

# train data
# Getting range of time for air quality data
min_date_train = AQ_data.time.min()
max_date_train = AQ_data.time.max()
# Creating DATE_HRS dataframe
Date_hrs_train = Date_hrs(min_date_train, max_date_train)
# since we need to train on all the air_quality data we using the same time frame for all the data

# Performing cartesian product of station and days_hrs dataframes
AQ_days_stations = pd.merge(Date_hrs_train.assign(key=0), AQ_station.assign(key=0), on='key').drop('key', axis=1)
print("Station * Days" + str(AQ_days_stations.shape))
# Joining the air_quality and days_stations dataframes
AQ_df = pd.merge(AQ_data, AQ_days_stations, how='right', left_on=['station_id','time'], 
                 right_on = ['station','date']).drop(['station_id','time'], axis =1)
print(AQ_df.columns)
AQ_df = AQ_df.sort_values(by=['date', 'station'], ascending=[True,True] )

# Making copy of new_df
AQ_nearstation = AQ_df
# Renaming all the columns of df_copy
AQ_nearstation = AQ_nearstation.drop(['nearest_AQ','nearest_OW','nearest_GW','longitude', 'latitude', 'type'], axis=1)
AQ_nearstation.rename(columns={'PM2.5': 'n_PM2.5','PM10': 'n_PM10', "NO2":"n_NO2","CO":"n_CO","O3":"n_O3",
                        "SO2":"n_SO2", "date":"n_date", "station":"n_station" }, inplace=True)
# Appending the new_df and df_copy based on nearest station and time
AQ_nearstation2 = pd.merge(AQ_df, AQ_nearstation, how='inner', left_on=['nearest_AQ','date'], 
                           right_on = ['n_station','n_date']).drop(['n_station','n_date', ], axis=1)
# Sorting by date and station
AQ_nearstation2 = AQ_nearstation2.sort_values(by=['station', 'date'], ascending=[True,True] )
# Sorting the index
AQ_nearstation2.sort_index(inplace= True)

# Filling the NaN values
# Finding the null values in each columns
print("Null values before preprocessing\n", AQ_nearstation2.applymap(lambda x: pd.isnull(x)).sum())
# since the zhiwuyuan_aq station has many missing values for more than a month we are assuming that the station is not 
# funcioning and we are filling it with zero
AQ_nearstation2.loc[AQ_nearstation2['station'] == "zhiwuyuan_aq",['PM2.5','PM10','NO2','CO','O3','SO2']] = 0
# Filling missing values with nearest station value
AQ_nearstation2['PM10'].fillna(AQ_nearstation2['n_PM10'], inplace=True)
AQ_nearstation2['PM2.5'].fillna(AQ_nearstation2['n_PM2.5'], inplace=True)
AQ_nearstation2['NO2'].fillna(AQ_nearstation2['n_NO2'], inplace=True)
AQ_nearstation2['CO'].fillna(AQ_nearstation2['n_CO'], inplace=True)
AQ_nearstation2['O3'].fillna(AQ_nearstation2['n_O3'], inplace=True)
AQ_nearstation2['SO2'].fillna(AQ_nearstation2['n_SO2'], inplace=True)
AQ_nearstation2 = ymd(AQ_nearstation2)
# Filling missing values in AQ data with the previous hour data of particular station, year, week
AQ_nearstation2[['PM2.5','PM10','NO2','CO','O3','SO2']] = AQ_nearstation2.groupby(["station","year","week","hour"])[['PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2']].transform(lambda x: x.ffill().bfill())
# # Filling missing values with mean of station , month, hour
AQ_nearstation2[['PM2.5','PM10','NO2','CO','O3','SO2']] = AQ_nearstation2.groupby(["station","hour"])[['PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2']].transform(lambda x: x.fillna(x.mean()))
print("Null values after preprocessing\n", AQ_nearstation2.applymap(lambda x:pd.isnull(x)).sum())
AQ_final = AQ_nearstation2.drop(['n_PM2.5', 'n_PM10', 'n_NO2', 'n_CO', 'n_O3','n_SO2', 'year','week', 'day', 'nearest_AQ'],axis=1)

###############################################################################################################################################################################################################################################################

#OBSERVED WEATHER DATA

# reading observed weather data
obs_wea_jan17_jan18 = pd.read_csv("observedWeather_201701-201801.csv")
obs_wea_feb18_mar18 = pd.read_csv("observedWeather_201802-201803.csv")
obs_wea_apr18 = pd.read_csv("observedWeather_201804.csv")
obs_wea_apr18.columns = ['id','station_id', 'utc_time', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed', 'wind_direction']
#appending the all observed weather data
OW_data = pd.DataFrame()
OW_data = obs_wea_jan17_jan18.append([obs_wea_feb18_mar18, obs_wea_apr18])
OW_data = OW_data.drop(["id","weather","latitude","longitude"], axis =1)
OW_data = OW_data.rename(columns={"utc_time": "time"})
OW_data["time"] = pd.to_datetime(OW_data['time'], format='%Y-%m-%d %H:%M:%S')
# Checking the shape of AQ_data before removing duplicates
print("Shape of OW_data before removing the duplicates:", OW_data.shape)
# Dropping duplicates
OW_data.drop_duplicates(subset= None, keep= "first", inplace= True)
# Sorting the entire dataframe based on station_id and time
OW_data = OW_data.sort_values(by=['station_id', 'time'], ascending=[True,True] )
# Checking the shape of OW_data after removing duplicates
print("Shape of OW_data after removing the duplicates:", OW_data.shape)

# Performing cartesian product of station and days_hrs dataframes
OW_days_stations = pd.merge(Date_hrs_train.assign(key=0), OW_station.assign(key=0), on='key').drop('key', axis=1)
print("Station * Days" + str(AQ_days_stations.shape))
# Joining the air_quality and days_stations dataframes
OW_df = pd.merge(OW_data, OW_days_stations, how='right', left_on=['station_id','time'], 
                 right_on = ['station','date']).drop(['station_id','time'], axis =1)
OW_df = OW_df.sort_values(by=['date', 'station'], ascending=[True,True] )

# Making copy of new_df
OW_nearstation = OW_df
# Renaming all the columns of df_copy
OW_nearstation = OW_nearstation.drop(['nearest_OW','longitude', 'latitude'], axis=1)
OW_nearstation.rename(columns={'humidity': 'n_humidity','pressure': 'n_pressure', "temperature":"n_temperature",
                               "wind_direction":"n_wind_direction","date":"n_date", "wind_speed":"n_wind_speed", 
                               "station":"n_station"}, inplace=True)
# Appending the new_df and df_copy based on nearest station and time
OW_nearstation2 = pd.merge(OW_df, OW_nearstation, how='inner', left_on=['nearest_OW','date'], 
                           right_on = ['n_station','n_date']).drop(['n_station','n_date'], axis=1)
# Sorting by date and station
OW_nearstation2 = OW_nearstation2.sort_values(by=['station', 'date'], ascending=[True,True] )
# Sorting the index
OW_nearstation2.sort_index(inplace= True)

# Filling the NaN values

# Finding the null values in each columns
print("Null values before preprocessing\n", OW_nearstation2.applymap(lambda x: pd.isnull(x)).sum())

# Filling missing values with nearest station value
OW_nearstation2['humidity'].fillna(OW_nearstation2['n_humidity'], inplace=True)
OW_nearstation2['pressure'].fillna(OW_nearstation2['n_pressure'], inplace=True)
OW_nearstation2['temperature'].fillna(OW_nearstation2['n_temperature'], inplace=True)
OW_nearstation2['wind_direction'].fillna(OW_nearstation2['n_wind_direction'], inplace=True)
OW_nearstation2['wind_speed'].fillna(OW_nearstation2['n_wind_speed'], inplace=True)
OW_nearstation2 = ymd(OW_nearstation2)
# filling missing values in OW data with the previous hour data of particular station, year, week
OW_nearstation2[['humidity','pressure','temperature','wind_direction','wind_speed']] = OW_nearstation2.groupby(["station","year","week","hour"])[['humidity','pressure','temperature','wind_direction','wind_speed']].transform(lambda x: x.ffill().bfill())
# Filling missing values with mean of station , month, hour
OW_nearstation2[['humidity','pressure','temperature','wind_direction','wind_speed']] = OW_nearstation2.groupby(["station","hour"])[['humidity','pressure','temperature','wind_direction','wind_speed']].transform(lambda x: x.fillna(x.mean()))
print("Null values after preprocessing\n", OW_nearstation2.applymap(lambda x:pd.isnull(x)).sum())
OW_data_final = OW_nearstation2.drop(['nearest_OW', 'n_humidity', 'n_pressure',
       'n_temperature', 'n_wind_direction', 'n_wind_speed', 'year','week', 'day'],axis=1)

###############################################################################################################################################################################################################################################################

# GRID WEATHER DATA
gw_2017 = pd.read_csv("gridWeather_201701-201803.csv") 
gw_2018 = pd.read_csv("gridWeather_201804.csv")
# renaming some columns
gw_2017 = gw_2017.rename(columns={"utc_time":"time","wind_speed/kph":"wind_speed","stationName":"station_id"})
# dropping the necessary columns
gw_2018 = gw_2018.drop(['id','weather'], axis =1)
# appending all the GW_data
GW_data = pd.concat([gw_2017,gw_2018], axis =0)
GW_data = GW_data.drop(["latitude","longitude"], axis =1)
# converting wind speed from m/s to Km/h
GW_data['wind_speed'] = GW_data.wind_speed*0.625
GW_data["time"] = pd.to_datetime(GW_data['time'], format='%Y-%m-%d %H:%M:%S')
# Checking the shape of AQ_data before removing duplicates
print("Shape of GW_data before removing the duplicates:", GW_data.shape)
# Dropping duplicates
GW_data.drop_duplicates(subset= None, keep= "first", inplace= True)
# Sorting the entire dataframe based on station_id and time
GW_data = GW_data.sort_values(by=['station_id', 'time'], ascending=[True,True] )
# Checking the shape of OW_data after removing duplicates
print("Shape of GW_data after removing the duplicates:", GW_data.shape)

# Performing cartesian product of GW_station and Date_hrs_train dataframes
GW_days_stations = pd.merge(Date_hrs_train.assign(key=0), GW_station.assign(key=0), on='key').drop('key', axis=1)
print("Station * Days" + str(GW_days_stations.shape))
# Joining the OW and days_stations dataframes
GW_df = pd.merge(GW_data, GW_days_stations, how='right', left_on=['station_id','time'], 
                 right_on = ['station','date']).drop(['station_id','time'], axis =1)
GW_df = GW_df.sort_values(by=['date', 'station'], ascending=[True,True] )

# Making copy of new_df
GW_nearstation = GW_df
# Renaming all the columns of df_copy
GW_nearstation = GW_nearstation.drop(['nearest_GW','longitude', 'latitude'], axis=1)
GW_nearstation.rename(columns={'humidity': 'n_humidity','pressure': 'n_pressure', "temperature":"n_temperature",
                               "wind_direction":"n_wind_direction","date":"n_date", "wind_speed":"n_wind_speed", 
                               "station":"n_station"}, inplace=True)
# Appending the new_df and df_copy based on nearest station and time
GW_nearstation2 = pd.merge(GW_df, GW_nearstation, how='inner', left_on=['nearest_GW','date'], 
                           right_on = ['n_station','n_date']).drop(['n_station','n_date'], axis=1)
# Sorting by date and station
GW_nearstation2 = GW_nearstation2.sort_values(by=['station', 'date'], ascending=[True,True] )
# Sorting the index
GW_nearstation2.sort_index(inplace= True)

# Filling the NaN values
# Finding the null values in each columns
print("Null values before preprocessing\n", GW_nearstation2.applymap(lambda x: pd.isnull(x)).sum())
# Filling missing values with nearest station value
GW_nearstation2['humidity'].fillna(GW_nearstation2['n_humidity'], inplace=True)
GW_nearstation2['pressure'].fillna(GW_nearstation2['n_pressure'], inplace=True)
GW_nearstation2['temperature'].fillna(GW_nearstation2['n_temperature'], inplace=True)
GW_nearstation2['wind_direction'].fillna(GW_nearstation2['n_wind_direction'], inplace=True)
GW_nearstation2['wind_speed'].fillna(GW_nearstation2['n_wind_speed'], inplace=True)
GW_nearstation2 = ymd(GW_nearstation2)
# filling missing values in OW data with the previous hour data of particular station, year, week
GW_nearstation2[['humidity','pressure','temperature','wind_direction','wind_speed']] = GW_nearstation2.groupby(["station","year","week","hour"])[['humidity','pressure','temperature','wind_direction','wind_speed']].transform(lambda x: x.ffill().bfill())
# Filling missing values with mean of station , month, hour
GW_nearstation2[['humidity','pressure','temperature','wind_direction','wind_speed']] = GW_nearstation2.groupby(["station","hour"])[['humidity','pressure','temperature','wind_direction','wind_speed']].transform(lambda x: x.fillna(x.mean()))
print("Null values after preprocessing\n", OW_nearstation2.applymap(lambda x:pd.isnull(x)).sum())
GW_final = GW_nearstation2.drop(['nearest_GW', 'n_humidity', 'n_pressure',
       'n_temperature', 'n_wind_direction', 'n_wind_speed', 'year','week', 'day'],axis=1)

###############################################################################################################################################################################################################################################################

# getting range of time
print("start_date: ",AQ_final.date.min(),"end_date",AQ_final.date.max())
print("start_date: ",OW_data_final.date.min(),"end_date",OW_data_final.date.max())
print("start_date: ",GW_final.date.min(),"end_date",GW_final.date.max())
# Dropping the unwanted column in AQ_final, OW_final, GW_final
AQ = AQ_final.drop(['longitude','latitude','type','NO2','CO','SO2'], axis=1)
OW = OW_data_final.drop(['month','hour','latitude','longitude'], axis =1)
GW = GW_final.drop(['latitude','longitude','month','hour'], axis=1)
df = AQ.copy()
# merging AQ_final and OW_final based on time and stationid
n_df = pd.merge(df, OW,  how='left', left_on=['nearest_OW','date'], 
                right_on = ['station','date'])
# Dropping the unwanted column in n_df
n_df = n_df.drop(["station_y"], axis=1)
n_df = n_df.rename(index=str, columns={"station_x": "station"})
# merging AQ_data and GW_data based on time and stationid
Final_df = pd.merge(n_df, GW,  how='left', left_on=['nearest_GW','date'], 
                right_on = ['station','date']
# Dropping the unwanted column in Final_df
Final_df = Final_df.drop(['station_y'], axis =1)
# Renaming the columns in Final_df
Final_df.rename(columns={'humidity_x': 'humidity_ow','pressure_x': 'pressure_ow', 
                        "temperature_x":"temperature_ow","wind_direction_x":"wind_direction_ow",
                        "wind_speed_x":"wind_speed_ow", 'humidity_y': 'humidity_gw',
                        'pressure_y': 'pressure_gw',"temperature_y":"temperature_gw",
                        "wind_direction_y":"wind_direction_gw","wind_speed_y":"wind_speed_gw",
                        "station_x":"station"}, inplace=True)
# Coping the Final_df to Final_data
Final_data = Final_df.copy()
# Replacing the noise with Nan
Final_data.loc[Final_data['humidity_ow'] >100,"humidity_ow"]= np.nan
Final_data.loc[Final_data['pressure_ow'] >2000,"pressure_ow"]= np.nan
Final_data.loc[Final_data['temperature_ow'] > 50,"temperature_ow"]= np.nan
Final_data.loc[Final_data['wind_speed_ow'] > 20,"wind_speed_ow"]= np.nan
Final_data.loc[Final_data['wind_direction_ow'] > 360,"wind_direction_ow"]= np.nan
# Filling NaN Values
Final_data['humidity_ow'].fillna(Final_data["humidity_gw"], inplace=True)
Final_data['pressure_ow'].fillna(Final_data["pressure_gw"], inplace=True)
Final_data['temperature_ow'].fillna(Final_data["temperature_gw"], inplace=True)
Final_data['wind_speed_ow'].fillna(Final_data["wind_speed_gw"], inplace=True)
Final_data['wind_direction_ow'].fillna(Final_data["wind_direction_gw"], inplace=True)
# sorting the final_dataset with respect to station and date
Final_data = Final_data.sort_values(by=['station', 'date'], ascending=[True,True] )
# taking only the grid weather data
X = Final_data[['station','humidity_gw','temperature_gw','wind_direction_gw','wind_speed_gw']].values
Y = Final_data[['PM2.5','PM10','O3']].values
#######################################################################################################################
#######################################################################################################################

# TEST DATA
# Getting the max and min time for test data
min_date_test = '2018-05-01 00:00:00'
max_date_test = '2018-05-02 23:00:00'
# Creating DATE_HRS dataframe
Date_hrs_test = Date_hrs(min_date_test, max_date_test)
# Performing cartesian product of OW_station and Date_hrs_train dataframes
AQ_test_days_stations = pd.merge(Date_hrs_test.assign(key=0), AQ_station.assign(key=0), on='key').drop('key', axis=1)
print("Station * Days" + str(AQ_test_days_stations.shape))

# OBSERVED WEATHE DATA
# reading observed weather data
OW_test = pd.read_csv("observedWeather_20180501-20180502.csv")
OW_test = OW_test.drop(["id","weather"],axis =1)
OW_test["time"] = pd.to_datetime(OW_test['time'], format='%Y-%m-%d %H:%M:%S')
# Sorting the entire dataframe based on station_id and time
OW_test = OW_test.sort_values(by=['station_id', 'time'], ascending=[True,True] )

# test data
# Getting range of time from May1 to May2
min_date_test = OW_test.time.min()
max_date_test = OW_test.time.max()

# Creating DATE_HRS dataframe
Date_hrs_test = Date_hrs(min_date_test, max_date_test)

# Performing cartesian product of OW_station and Date_hrs_train dataframes
OW_test_days_stations = pd.merge(Date_hrs_test.assign(key=0), OW_station.assign(key=0), on='key').drop('key', axis=1)
print("Station * Days" + str(OW_test_days_stations.shape))

# Joining the OW and days_stations dataframes
OW_df_test = pd.merge(OW_test, OW_test_days_stations, how='right', left_on=['station_id','time'], 
                 right_on = ['station','date']).drop(['station_id','time'], axis =1)
OW_df_test = OW_df_test.sort_values(by=['date', 'station'], ascending=[True,True] )

# GRID WEATHER DATA
# reading observed weather data
GW_test = pd.read_csv("gridWeather_20180501-20180502.csv")
GW_test = GW_test.drop(["id","weather"],axis =1)
GW_test["time"] = pd.to_datetime(GW_test['time'], format='%Y-%m-%d %H:%M:%S')
# Sorting the entire dataframe based on station_id and time
GW_test = GW_test.sort_values(by=['station_id', 'time'], ascending=[True,True] )
# Performing cartesian product of OW_station and Date_hrs_train dataframes
GW_test_days_stations = pd.merge(Date_hrs_test.assign(key=0), GW_station.assign(key=0), on='key').drop('key', axis=1)
print("Station * Days" + str(GW_test_days_stations.shape))
# Joining the OW and days_stations dataframes
GW_df_test = pd.merge(GW_test, GW_test_days_stations, how='right', left_on=['station_id','time'], 
                 right_on = ['station','date']).drop(['station_id','time'], axis =1)
GW_df_test = GW_df_test.sort_values(by=['date', 'station'], ascending=[True,True] )

# Merging all the test data
NEW_df = pd.merge(AQ_test_days_stations, OW_test,  how='left', left_on=['nearest_OW','date'], 
                right_on = ['station_id','time'])
Final_test_df = pd.merge(NEW_df, GW_test,  how='left', left_on=['nearest_GW','date'], 
                right_on = ['station_id','time'])
Final_test_df = Final_test_df.drop(['longitude', 'latitude', 'type', 'nearest_AQ',
       'nearest_OW', 'nearest_GW','station_id_x', 'time_x','station_id_y', 'time_y' ], axis =1)
Final_test_df.rename(columns={'humidity_x': 'humidity_ow','pressure_x': 'pressure_ow', 
                        "temperature_x":"temperature_ow","wind_direction_x":"wind_direction_ow",
                        "wind_speed_x":"wind_speed_ow", 'humidity_y': 'humidity_gw',
                        'pressure_y': 'pressure_gw',"temperature_y":"temperature_gw",
                        "wind_direction_y":"wind_direction_gw","wind_speed_y":"wind_speed_gw",
                        "station_x":"station"}, inplace=True)
#######################################################################################################################
#######################################################################################################################
# MODEL
# One hot encoding the station data
scaler = MinMaxScaler()
le = LabelEncoder()
X[:,0] = le.fit_transform(X[:,0])
ohe = OneHotEncoder(categorical_features=[0])
ohe_X = ohe.fit_transform(X).toarray()
X_scaled = scaler.fit_transform(ohe_X)
# Error Function
def smape(actual, predicted):
    dividend= np.abs(np.array(actual) - np.array(predicted))
    denominator = np.array(actual) + np.array(predicted)
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))
# Use Random Forest Regressor to predict the values
model_RF = RandomForestRegressor(random_state=42)
model_multi_xgb = MultiOutputRegressor(XGBRegressor(max_depth=10, learning_rate=0.1, n_estimators=100, \
                                       silent=True, objective='reg:linear', booster='gbtree', n_jobs=-1))
SMAPE_RF = []
SMAPE_XG = []
# Use K Fold Cross Validation to check the efficiency of the model
fold = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in fold.split(X_scaled):
    x_train, x_val = X_scaled[train_index], X_scaled[test_index]
    y_train, y_val = Y[train_index], Y[test_index]
    model_RF.fit(x_train, y_train)
    preds = model_RF.predict(x_val)
    SMAPE_RF.append(smape(y_val,preds))
print("Error for random forest: ",(sum(SMAPE_RF)/len(SMAPE_RF)))

# Prediction
TEST = Final_test_df[['station','humidity_gw','temperature_gw','wind_direction_gw','wind_speed_gw']].values
# One hot encode and normalize similair to train data
TEST[:,0] = le.fit_transform(TEST[:,0])
TEST_ohe = ohe.transform(TEST).toarray()
TEST_scaled = scaler.fit_transform(TEST_ohe)

predictions = model_RF.predict(TEST_scaled)

test_idx = Final_test_df[['station']]
index = list(range(0,48)) 
idx = cycle(index)
test_idx['index'] = [next(idx) for i in range(len(test_idx))]
test_idx['test_id'] = test_idx ['station']+'#'+test_idx['index'].astype(str)
test_idx.drop(['index','station'],axis=1, inplace=True)
test_idx1 = test_idx.values
output = np.concatenate((test_idx, predictions), axis=1)
np.savetxt('submission.csv', output, delimiter=',', header='test_id,PM2.5,PM10,O3', fmt='%s,%f,%f,%f', comments='')